In [5]:
import re, sys, dash, mpl_toolkits.basemap

import pandas as pd
import requests as r
import zipcodes as zc
import matplotlib.pyplot as plt
import dash_core_components as dcc
import dash_html_components as html

from bs4 import BeautifulSoup as bs
from dash.dependencies import Input, Output
from textwrap import dedent as d


In [14]:
df = pd.read_csv('/Users/tyler/Downloads/transactions (2).csv')
df.drop(columns=['Category','Labels', 'Notes'], inplace=True)

In [17]:
df

,Date,Description,Original Description,Amount,Transaction Type,Account Name
0,3/19/2018,Chipotle,CHIPOTLE 0114,10.59,debit,CREDIT CARD
1,2/28/2018,Chipotle,CHIPOTLE 0894,10.63,debit,CREDIT CARD
2,2/15/2018,Chipotle,CHIPOTLE 0894,18.26,debit,CREDIT CARD
3,1/25/2018,Chipotle,CHIPOTLE 0894,12.86,debit,CREDIT CARD
4,1/25/2018,Chipotle,CHIPOTLE 0894,5.45,debit,CREDIT CARD
5,1/07/2018,Chipotle,CHIPOTLE 0894,10.79,debit,CREDIT CARD
6,10/23/2017,Chipotle,CHIPOTLE 1702,15.29,debit,CREDIT CARD
7,4/28/2017,Chipotle,CHIPOTLE 1145,9.99,debit,CREDIT CARD
8,4/17/2017,Chipotle,CHIPOTLE 1702,10.04,debit,CREDIT CARD
9,4/03/2017,Chipotle,Check Card: CHIPOTLE 1460 SANFRANCISCO CA 04/...,12.00,debit,PLUS CHECKING


In [72]:
df['store_no'] = df['Original Description'].apply(parse_store_loc)
cdf = df
cdf.head()

,Date,Description,Original Description,Amount,Transaction Type,Account Name,store_no
0,3/19/2018,Chipotle,CHIPOTLE 0114,10.59,debit,CREDIT CARD,0114
1,2/28/2018,Chipotle,CHIPOTLE 0894,10.63,debit,CREDIT CARD,0894
2,2/15/2018,Chipotle,CHIPOTLE 0894,18.26,debit,CREDIT CARD,0894
3,1/25/2018,Chipotle,CHIPOTLE 0894,12.86,debit,CREDIT CARD,0894
4,1/25/2018,Chipotle,CHIPOTLE 0894,5.45,debit,CREDIT CARD,0894


In [103]:
#zip_addr = cdf.store_no.apply(search_chip_no)
#zip_df = pd.DataFrame(list(zip_addr), columns=['store_no','zip_code','address'])
zip_df.head()
print(len(zip_df))
print(len(cdf))

76
76


In [119]:
#tdf = pd.concat([cdf, zip_df], ignore_index=True)
tdf = cdf.join(zip_df, rsuffix='_y')

len(tdf)

76

In [121]:
tdf.drop('store_no_y', inplace=True, axis = 1)

In [123]:
len(tdf)

76

In [124]:
locs = None
locs = tdf.zip_code.apply(pull_loc_info)
locs_df = pd.DataFrame(list(locs),columns=['lat', 'lon','city','state'])
ldf = pd.concat([tdf, locs_df], axis=1)

In [125]:
len(ldf)

76

In [126]:
ldf.head()

,Date,Description,Original Description,Amount,Transaction Type,Account Name,store_no,zip_code,address,lat,lon,city,state
0,3/19/2018,Chipotle,CHIPOTLE 0114,10.59,debit,CREDIT CARD,0114,80027,0114 - Louisville-(00114) - 375 S. McCaslin Bl...,39.96,-105.14,LOUISVILLE,CO
1,2/28/2018,Chipotle,CHIPOTLE 0894,10.63,debit,CREDIT CARD,0894,80301,"0894 - 29th Street-(00894) - 1650 28th St, Ste...",40.02,-105.25,BOULDER,CO
2,2/15/2018,Chipotle,CHIPOTLE 0894,18.26,debit,CREDIT CARD,0894,80301,"0894 - 29th Street-(00894) - 1650 28th St, Ste...",40.02,-105.25,BOULDER,CO
3,1/25/2018,Chipotle,CHIPOTLE 0894,12.86,debit,CREDIT CARD,0894,80301,"0894 - 29th Street-(00894) - 1650 28th St, Ste...",40.02,-105.25,BOULDER,CO
4,1/25/2018,Chipotle,CHIPOTLE 0894,5.45,debit,CREDIT CARD,0894,80301,"0894 - 29th Street-(00894) - 1650 28th St, Ste...",40.02,-105.25,BOULDER,CO


In [28]:
def pull_span_zip(span_text:str, store_no:str):
    pat = '^%s'%(store_no)
    if re.match(pat, span_text):
        zipcode = re.findall(' (\d{5})', span_text)
        return zipcode[0]
    else:
        return None
    
        
def search_chip_no(store_no:str):
    chip_job_url = "https://jobs.chipotle.com/search-jobs/%s"%(str(store_no))
    cont = bs(r.get(chip_job_url).content, 'html5lib')
    spans = cont.find_all('span')
    lines = [span.get_text() for span in spans]
    for x in lines: 
        zipc = pull_span_zip(x, store_no)
        if zipc:
            break
    
    return [store_no, zipc, x]
            
def chip_txns(df:pd.DataFrame, regex):
    df = df[df['Original Description'].str.contains(regex)]
    return df

def parse_store_loc(descriptor:str):
    nos = re.findall(' (\d{4})', descriptor)
    return nos[0]

def pull_lat_long(zipcode:str):
    try:
        return [zc.matching(zipcode)[0]['lat'],
                  zc.matching(zipcode)[0]['long']]
    except:
        return [None,None]
    
def pull_loc_info(zipcode:str):
    try:
        zcmatch = zc.matching(zipcode)[0]
        return [zcmatch['lat'],
                  zcmatch['long'],
                  zcmatch['city'],
                  zcmatch['state']
                  ]
    except:
        return [None]*4

In [50]:
# LIMIT TO CHIPOTLE TXNS
cdf = chip_txns(df, 'CHIPOTLE')
# EXTRACT STORE NO
cdf['store_no'] = cdf.loc[:,'Original Description'].apply(lambda x: parse_store_loc(x))
# RESET INDEX
cdf.reset_index(inplace=True, drop=True)

# Pull Location Info
zip_addr = cdf.store_no.apply(search_chip_no)
zip_df = pd.DataFrame(list(zip_addr), columns=['store_no','zip_code','address'])
# CONCAT zip_df cols to cdf
tdf = pd.concat([cdf, zip_df], axis=1)

# GET LAT LONS
latlons = tdf.zip_code.apply(pull_lat_long)
latlons_df = pd.DataFrame(list(latlons),columns=['lat', 'long'])
# CONCAT latlons df cols to cdf
ldf = pd.concat([tdf, latlons_df], axis=1)

In [47]:
ldf.head()

,Date,Description,Original Description,Amount,Transaction Type,Account Name,store_no,zip_code,address,lat,lon,city,state
0,3/19/2018,Chipotle,CHIPOTLE 0114,10.59,debit,CREDIT CARD,0114,80027,0114 - Louisville-(00114) - 375 S. McCaslin Bl...,39.96,-105.14,LOUISVILLE,CO
1,2/28/2018,Chipotle,CHIPOTLE 0894,10.63,debit,CREDIT CARD,0894,80301,"0894 - 29th Street-(00894) - 1650 28th St, Ste...",40.02,-105.25,BOULDER,CO
2,2/28/2018,Chipotle,CHIPOTLE 0894,10.63,debit,CREDIT CARD,0894,80301,"0894 - 29th Street-(00894) - 1650 28th St, Ste...",40.02,-105.25,BOULDER,CO
3,2/28/2018,Chipotle,CHIPOTLE 0894,10.63,debit,CREDIT CARD,0894,80301,"0894 - 29th Street-(00894) - 1650 28th St, Ste...",40.02,-105.25,BOULDER,CO
4,2/28/2018,Chipotle,CHIPOTLE 0894,10.63,debit,CREDIT CARD,0894,80301,"0894 - 29th Street-(00894) - 1650 28th St, Ste...",40.02,-105.25,BOULDER,CO


In [52]:
len(ldf)

604

In [133]:
df_piv = ldf.groupby(['store_no','lat','lon', 'state', 'address']).agg({'Amount':['sum','count','mean']}).reset_index()

In [134]:
df_piv

store_no    lat     lon state  \
                                   
0      0043  30.26  -97.74    TX   
1      0054  39.76 -104.87    CO   
2      0056  30.26  -97.74    TX   
3      0114  39.96 -105.14    CO   
4      0436  40.71  -73.99    NY   
5      0437  45.51 -122.67    OR   
6      0839  37.78 -122.39    CA   
7      0894  40.02 -105.25    CO   
8      1102  37.67 -122.47    CA   
9      1145  39.69 -104.88    CO   
10     1174  37.49 -122.26    CA   
11     1187  37.68 -122.43    CA   
12     1230  37.77 -122.41    CA   
13     1265  37.41 -122.13    CA   
14     1460  37.77 -122.41    CA   
15     1611  37.38 -122.08    CA   
16     1617  37.37 -122.18    CA   
17     1637  37.72 -122.48    CA   
18     1702  37.77 -122.41    CA   
19     1738  37.81 -122.24    CA   
20     2080  37.72 -122.48    CA   
21     2341  42.35  -71.06    MA   
22     2345  36.11 -115.12    NV   

                                              address  Amount                   
                                                          sum count       mean  
0   0043 - Westbank-(00043) - 3300 Bee Cave Road, ...   18.03     2   9.015000  
1   0054 - 6th & Broadway-(00054) - 550 Broadway, ...    7.78     1   7.780000  
2   0056 - 45th & Lamar-(00056) - 4400 N. Lamar Bl...    7.20     1   7.200000  
3   0114 - Louisville-(00114) - 375 S. McCaslin Bl...   10.59     1  10.590000  
4   0436 - Grand Central-(00436) - 150 E 44th St, ...   11.00     1  11.000000  
5   0437 - Director Building-(00437) - 240 SW Yamh...   12.00     1  12.000000  
6   0839 - 525 Market Street-(00839) - 525 Market ...   27.18     2  13.590000  
7   0894 - 29th Street-(00894) - 1650 28th St, Ste...   68.29     6  11.381667  
8   1102 - Daly City-(01102) - 213 Westlake Center...   11.61     1  11.610000  
9   1145 - London Square Relo-(01145) - 2300 S Par...    9.99     1   9.990000  
10  1174 - San Carlos-(01174) - 1135 Industrial Ro...   18.38     2   9.190000  
11  1187 - Colma-(01187) - 990 Serramonte Blvd, Su...  109.15    11   9.922727  
12  1230 - Union Square-(01230) - 232 O'Farrell St...    7.56     1   7.560000  
13  1265 - Palo Alto-(01265) - 2675 El Camino Real...   43.95     5   8.790000  
14  1460 - Metreon-(01460) - 121 4th St #135, San ...   29.08     3   9.693333  
15  1611 - El Monte Center-(01611) - 1039 A El Mon...   16.32     2   8.160000  
16  1617 - Stanford Center-(01617) - 180 El Camino...   10.17     1  10.170000  
17  1637 - Lakeshore Plaza-(01637) - 1523 Sloat Bl...   61.81     6  10.301667  
18  1702 - San Francisco Centre-(01702) - 865 Mark...  205.64    18  11.424444  
19  1738 - Oakland Lakeshore-(01738) - 3271 Lakesh...   12.26     1  12.260000  
20  2080 - Stonestown Galleria-(02080) - 3251 20th...   33.32     3  11.106667  
21  2341 - 553 Boylston Street-(02341) - 553 Boyls...   13.00     1  13.000000  
22  2345 - Miracle Mile Shops-(02345) - 3663 Las V...   13.09     1  13.090000

In [152]:
amounts = df_piv['Amount']
df_piv['text'] = 'Visits: '+amounts['count'].map(str)+\
                  '\nAvg $: '+amounts['mean'].map(str)+\
                  '\nTotal $: '+amounts['sum'].map(str)+\
                  '\n'+df_piv['address']

In [161]:
list(df_piv.state.unique())

['TX', 'CO', 'NY', 'OR', 'CA', 'MA', 'NV']

In [162]:
{k:k for k in list(df_piv.state.unique())}

{'CA': 'CA',
 'CO': 'CO',
 'MA': 'MA',
 'NV': 'NV',
 'NY': 'NY',
 'OR': 'OR',
 'TX': 'TX'}

In [164]:
[{'label':k, 'value': k} for k in list(df_piv.state.unique())]


[{'label': 'TX', 'value': 'TX'},
 {'label': 'CO', 'value': 'CO'},
 {'label': 'NY', 'value': 'NY'},
 {'label': 'OR', 'value': 'OR'},
 {'label': 'CA', 'value': 'CA'},
 {'label': 'MA', 'value': 'MA'},
 {'label': 'NV', 'value': 'NV'}]

In [171]:
df_piv.Amount['sum'].sum()


757.4000000000001

In [172]:
df_piv

store_no    lat     lon state  \
                                   
0      0043  30.26  -97.74    TX   
1      0054  39.76 -104.87    CO   
2      0056  30.26  -97.74    TX   
3      0114  39.96 -105.14    CO   
4      0436  40.71  -73.99    NY   
5      0437  45.51 -122.67    OR   
6      0839  37.78 -122.39    CA   
7      0894  40.02 -105.25    CO   
8      1102  37.67 -122.47    CA   
9      1145  39.69 -104.88    CO   
10     1174  37.49 -122.26    CA   
11     1187  37.68 -122.43    CA   
12     1230  37.77 -122.41    CA   
13     1265  37.41 -122.13    CA   
14     1460  37.77 -122.41    CA   
15     1611  37.38 -122.08    CA   
16     1617  37.37 -122.18    CA   
17     1637  37.72 -122.48    CA   
18     1702  37.77 -122.41    CA   
19     1738  37.81 -122.24    CA   
20     2080  37.72 -122.48    CA   
21     2341  42.35  -71.06    MA   
22     2345  36.11 -115.12    NV   

                                              address  Amount        \
                                                          sum count   
0   0043 - Westbank-(00043) - 3300 Bee Cave Road, ...   18.03     2   
1   0054 - 6th & Broadway-(00054) - 550 Broadway, ...    7.78     1   
2   0056 - 45th & Lamar-(00056) - 4400 N. Lamar Bl...    7.20     1   
3   0114 - Louisville-(00114) - 375 S. McCaslin Bl...   10.59     1   
4   0436 - Grand Central-(00436) - 150 E 44th St, ...   11.00     1   
5   0437 - Director Building-(00437) - 240 SW Yamh...   12.00     1   
6   0839 - 525 Market Street-(00839) - 525 Market ...   27.18     2   
7   0894 - 29th Street-(00894) - 1650 28th St, Ste...   68.29     6   
8   1102 - Daly City-(01102) - 213 Westlake Center...   11.61     1   
9   1145 - London Square Relo-(01145) - 2300 S Par...    9.99     1   
10  1174 - San Carlos-(01174) - 1135 Industrial Ro...   18.38     2   
11  1187 - Colma-(01187) - 990 Serramonte Blvd, Su...  109.15    11   
12  1230 - Union Square-(01230) - 232 O'Farrell St...    7.56     1   
13  1265 - Palo Alto-(01265) - 2675 El Camino Real...   43.95     5   
14  1460 - Metreon-(01460) - 121 4th St #135, San ...   29.08     3   
15  1611 - El Monte Center-(01611) - 1039 A El Mon...   16.32     2   
16  1617 - Stanford Center-(01617) - 180 El Camino...   10.17     1   
17  1637 - Lakeshore Plaza-(01637) - 1523 Sloat Bl...   61.81     6   
18  1702 - San Francisco Centre-(01702) - 865 Mark...  205.64    18   
19  1738 - Oakland Lakeshore-(01738) - 3271 Lakesh...   12.26     1   
20  2080 - Stonestown Galleria-(02080) - 3251 20th...   33.32     3   
21  2341 - 553 Boylston Street-(02341) - 553 Boyls...   13.00     1   
22  2345 - Miracle Mile Shops-(02345) - 3663 Las V...   13.09     1   

                                                            text  
         mean                                                     
0    9.015000  Visits: 2\nAvg $: 9.015\nTotal $: 18.03\n0043 ...  
1    7.780000  Visits: 1\nAvg $: 7.78\nTotal $: 7.78\n0054 - ...  
2    7.200000  Visits: 1\nAvg $: 7.2\nTotal $: 7.2\n0056 - 45...  
3   10.590000  Visits: 1\nAvg $: 10.59\nTotal $: 10.59\n0114 ...  
4   11.000000  Visits: 1\nAvg $: 11.0\nTotal $: 11.0\n0436 - ...  
5   12.000000  Visits: 1\nAvg $: 12.0\nTotal $: 12.0\n0437 - ...  
6   13.590000  Visits: 2\nAvg $: 13.59\nTotal $: 27.18\n0839 ...  
7   11.381667  Visits: 6\nAvg $: 11.381666666666668\nTotal $:...  
8   11.610000  Visits: 1\nAvg $: 11.61\nTotal $: 11.61\n1102 ...  
9    9.990000  Visits: 1\nAvg $: 9.99\nTotal $: 9.99\n1145 - ...  
10   9.190000  Visits: 2\nAvg $: 9.190000000000001\nTotal $: ...  
11   9.922727  Visits: 11\nAvg $: 9.922727272727272\nTotal $:...  
12   7.560000  Visits: 1\nAvg $: 7.56\nTotal $: 7.56\n1230 - ...  
13   8.790000  Visits: 5\nAvg $: 8.79\nTotal $: 43.9499999999...  
14   9.693333  Visits: 3\nAvg $: 9.693333333333333\nTotal $: ...  
15   8.160000  Visits: 2\nAvg $: 8.16\nTotal $: 16.32\n1611 -...  
16  10.170000  Visits: 1\nAvg $: 10.17\nTotal $: 10.17\n1617 ...  
17  10.301667  Visits: 6\nAvg $: 10.301666666666668\n